The data was obtained from the Accenture Virtual Internship, and the business question revolved around analyzing content categories to identify the top 5 categories with the highest popularity. The process involved various steps, such as Data Understanding, Data Cleaning, Data Modeling, and Data Analysis.

I went a step further by incorporating additional operational metrics, including the Number of Reactions per Category, Number of Reactions per Reaction Type, Sentiment Analysis, User Engagement, and Time-based Analysis.

#### Connect to database, Create table and Copy data From provided datasets

#### Data Understanding

In [1]:
%load_ext sql

In [2]:
%sql postgresql://postgres:111111@:5433/socialbuzz

In [3]:
%%sql DROP TABLE IF EXISTS contents CASCADE; 
    CREATE TABLE contents(
    content_id UUID PRIMARY KEY,
    user_id UUID,
    content_type VARCHAR(255),
    category VARCHAR(255),
    url VARCHAR(255)
);

 * postgresql://postgres:***@:5433/socialbuzz
Done.
Done.


[]

In [4]:
%%sql DROP TABLE IF EXISTS reactions CASCADE; 
    CREATE TABLE reactions(
    content_id UUID,
    user_id UUID,
    reaction_type VARCHAR(255),
    datetime TIMESTAMP, 
    FOREIGN KEY (content_id) 
        REFERENCES contents(content_id)
);

 * postgresql://postgres:***@:5433/socialbuzz
Done.
Done.


[]

In [5]:
%sql DROP TABLE IF EXISTS reaction_types CASCADE; \
    CREATE TABLE reaction_types(\
    reaction_type VARCHAR(255) NOT NULL,\
    sentiment VARCHAR(255) NOT NULL,\
    score smallint NOT NULL\
);

 * postgresql://postgres:***@:5433/socialbuzz
Done.
Done.


[]

In [6]:
%%sql COPY contents(content_id, 
                    user_id, 
                    content_type, 
                    category, 
                    url)
FROM 'C:\socialBuzz\content.csv'
DELIMITER ','
CSV HEADER;

 * postgresql://postgres:***@:5433/socialbuzz
1000 rows affected.


[]

In [7]:
%sql COPY reactions(content_id, user_id, reaction_type, datetime)\
FROM 'C:\socialBuzz\reactions.csv'\
DELIMITER ','\
CSV HEADER;

 * postgresql://postgres:***@:5433/socialbuzz
25553 rows affected.


[]

In [8]:
%sql COPY reaction_types(reaction_type, sentiment, score)\
FROM 'C:\socialBuzz\reactiontypes.csv'\
DELIMITER ','\
CSV HEADER;

 * postgresql://postgres:***@:5433/socialbuzz
16 rows affected.


[]

#### View Datasets

In [9]:
%sql SELECT * FROM contents LIMIT 5;

 * postgresql://postgres:***@:5433/socialbuzz
5 rows affected.


content_id,user_id,content_type,category,url
97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,photo,Studying,https://socialbuzz.cdn.com/content/storage/97522e57-d9ab-4bd6-97bf-c24d952602d2
9f737e0a-3cdd-4d29-9d24-753f4e3be810,beb1f34e-7870-46d6-9fc7-2e12eb83ce43,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/9f737e0a-3cdd-4d29-9d24-753f4e3be810
230c4e4d-70c3-461d-b42c-ec09396efb3f,a5c65404-5894-4b87-82f2-d787cbee86b4,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/230c4e4d-70c3-461d-b42c-ec09396efb3f
356fff80-da4d-4785-9f43-bc1261031dc6,9fb4ce88-fac1-406c-8544-1a899cee7aaf,photo,technology,https://socialbuzz.cdn.com/content/storage/356fff80-da4d-4785-9f43-bc1261031dc6
01ab84dd-6364-4236-abbb-3f237db77180,e206e31b-5f85-4964-b6ea-d7ee5324def1,video,food,https://socialbuzz.cdn.com/content/storage/01ab84dd-6364-4236-abbb-3f237db77180


In [10]:
%sql SELECT * FROM reactions LIMIT 5;

 * postgresql://postgres:***@:5433/socialbuzz
5 rows affected.


content_id,user_id,reaction_type,datetime
97522e57-d9ab-4bd6-97bf-c24d952602d2,None,None,2021-04-22 15:17:00
97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust,2020-11-07 09:43:00
97522e57-d9ab-4bd6-97bf-c24d952602d2,92b87fa5-f271-43e0-af66-84fac21052e6,dislike,2021-06-17 12:22:00
97522e57-d9ab-4bd6-97bf-c24d952602d2,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared,2021-04-18 05:13:00
97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust,2021-01-06 19:13:00


In [11]:
%sql SELECT * FROM reaction_types LIMIT 5;

 * postgresql://postgres:***@:5433/socialbuzz
5 rows affected.


reaction_type,sentiment,score
heart,positive,60
want,positive,70
disgust,negative,0
hate,negative,5
interested,positive,30


#### Data Cleaning:

- Remove rows that have missing values
- Standarize values in certain column when neccessary. 

Validate rows with null values

In [12]:
%sql SELECT\
    COUNT(CASE WHEN content_id IS NULL THEN 1 END) AS null_count_content_id,\
    COUNT(CASE WHEN user_id IS NULL THEN 1 END) AS null_count_user_id,\
    COUNT(CASE WHEN reaction_type IS NULL THEN 1 END) AS null_count_reaction_type,\
    COUNT(CASE WHEN datetime IS NULL THEN 1 END) AS null_count_datetime\
FROM reactions;

 * postgresql://postgres:***@:5433/socialbuzz
1 rows affected.


null_count_content_id,null_count_user_id,null_count_reaction_type,null_count_datetime
0,3019,980,0


There are no null values in the "contents" and "reaction_types" tables. However, missing values were identified in the "user_id" and "reaction_type" columns within the "reactions" table. It is crucial to note that while missing values in the "reaction_type" column may impact our results, the absence of values in the "user_id" column will not affect any outcomes. As a result, the rows with missing values in the "user_id" column will be left unchanged, while the rows with missing values in the "reaction_types" column will be dropped.

In [13]:
%sql SELECT category, COUNT(*) \
FROM contents \
WHERE category ILIKE '%animals%'\
GROUP BY category \
ORDER BY category;

 * postgresql://postgres:***@:5433/socialbuzz
3 rows affected.


category,count
"""animals""",1
animals,67
Animals,4


In [14]:
%sql \
SELECT \
    REPLACE(LOWER(category), '\"', '') AS cleaned_category, \
    COUNT(*) \
FROM \
    contents \
GROUP BY \
    cleaned_category \
ORDER BY \
    cleaned_category;

 * postgresql://postgres:***@:5433/socialbuzz
16 rows affected.


cleaned_category,count
animals,72
cooking,61
culture,67
dogs,58
education,59
fitness,66
food,64
healthy eating,62
public speaking,50
science,68


In [15]:
%sql DROP VIEW IF EXISTS n_contents CASCADE; \
CREATE VIEW n_contents AS \
SELECT content_id, content_type, REPLACE(LOWER(category), '\"', '') AS cleaned_category, url  \
FROM contents;

 * postgresql://postgres:***@:5433/socialbuzz
Done.
Done.


[]

In [16]:
%sql SELECT * \
FROM n_contents \
LIMIT 5;

 * postgresql://postgres:***@:5433/socialbuzz
5 rows affected.


content_id,content_type,cleaned_category,url
97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,studying,https://socialbuzz.cdn.com/content/storage/97522e57-d9ab-4bd6-97bf-c24d952602d2
9f737e0a-3cdd-4d29-9d24-753f4e3be810,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/9f737e0a-3cdd-4d29-9d24-753f4e3be810
230c4e4d-70c3-461d-b42c-ec09396efb3f,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/230c4e4d-70c3-461d-b42c-ec09396efb3f
356fff80-da4d-4785-9f43-bc1261031dc6,photo,technology,https://socialbuzz.cdn.com/content/storage/356fff80-da4d-4785-9f43-bc1261031dc6
01ab84dd-6364-4236-abbb-3f237db77180,video,food,https://socialbuzz.cdn.com/content/storage/01ab84dd-6364-4236-abbb-3f237db77180


#### Data Analysis

#### Popularity Score by Category
- Calculate the aggregate popularity score for each category by summing up the scores of all reactions within that category.
- Identify the top 5 categories with the highest aggregate popularity scores.
#### Number of Reactions per Category
- Count the number of reactions for each category.
- This metric helps to understand not just the popularity score but also the overall engagement level for each category.

The Popularity Score and the Number of Reactions by category exhibit a parallel trend, indicating a consistent pattern. The top 5 categories in both the Popularity Score and the Number of Reactions are animals, science, healthy eating, technology, and food.

In [17]:
%%sql 
SELECT popularity.category, popularity.popularity_score, 
reaction.number_of_reaction
FROM(
SELECT cleaned_category AS category, 
SUM(score) AS popularity_score
FROM reactions AS r
JOIN n_contents AS n
ON n.content_id = r.content_id
JOIN reaction_types AS rt
ON rt.reaction_type = r.reaction_type
GROUP BY category) AS popularity
JOIN (
    SELECT cleaned_category AS category, 
    COUNT(datetime) AS number_of_reaction
    FROM reactions AS r
    JOIN n_contents AS n
    ON n.content_id = r.content_id
    GROUP BY category
) AS reaction
ON reaction.category = popularity.category
ORDER BY popularity_score DESC;

 * postgresql://postgres:***@:5433/socialbuzz
16 rows affected.


category,popularity_score,number_of_reaction
animals,74965,1967
science,71168,1864
healthy eating,69339,1779
technology,68738,1769
food,66676,1762
culture,66579,1741
travel,64880,1716
cooking,64756,1724
soccer,57783,1520
education,57436,1492


#### Number of Reaction per Reaction Type
- Calculate the count of reaction for each reaction type.
- This can provide insights into which types of reactions are generally more popular.

The top 5 reaction_types are heart(positive), scared(negative), peeking(neautral), hate(negative) and interested(positive). 

In [18]:
%%sql 
SELECT r.reaction_type, sentiment,
COUNT(score)
FROM reactions AS r
JOIN reaction_types AS rt
ON r.reaction_type = rt.reaction_type
GROUP BY r.reaction_type, sentiment
ORDER BY count DESC;

 * postgresql://postgres:***@:5433/socialbuzz
16 rows affected.


reaction_type,sentiment,count
heart,positive,1622
scared,negative,1572
peeking,neutral,1559
hate,negative,1552
interested,positive,1549
dislike,negative,1548
adore,positive,1548
want,positive,1539
love,positive,1534
disgust,negative,1526


#### Sentiment Analysis:

- Analyze the sentiment associated with each reaction type within a category.
- Understand whether positive, negative, or neutral sentiments contribute more to the overall popularity. 

Out of the total reaction count, positive sentiments made the most significant contribution. However, it's important to note that positive sentiments were expressed through 9 reaction types, while negative sentiments were conveyed through 5 reactions, and neutral sentiments only had 2. To better gauge sentiment, dividing the total count byy reaction made by the number of reaction types provides a more appropriate measure.

In the end, the average count for all three sentiments is similar across the board.

In [19]:
%%sql SELECT sentiment, 
COUNT(r.reaction_type) AS count,
COUNT(DISTINCT r.reaction_type) AS distinct_count,
COUNT(r.reaction_type) / COUNT(DISTINCT r.reaction_type) 
AS average_count
FROM reactions AS r
JOIN reaction_types AS rt
ON r.reaction_type = rt.reaction_type
GROUP BY sentiment
ORDER BY count DESC;

 * postgresql://postgres:***@:5433/socialbuzz
3 rows affected.


sentiment,count,distinct_count,average_count
positive,13807,9,1534
negative,7695,5,1539
neutral,3071,2,1535


#### User Engagement:

- Calculate the average number of reactions per user.
- This metric helps to understand how engaged users are with the content across different categories.


The dataset monitored a total of 500 user_ids. On average, each user_id contributed 49 reactions. The user_id with the highest number of reactions provided a total of 65 reactions.

In [20]:
%%sql

SELECT COUNT(DISTINCT user_id) AS user_count,
FROM reactions;

 * postgresql://postgres:***@:5433/socialbuzz
(psycopg2.errors.SyntaxError) syntax error at or near "FROM"
LINE 2: FROM reactions;
        ^

[SQL: SELECT COUNT(DISTINCT user_id) AS user_count,
FROM reactions;]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [21]:
%%sql 
SELECT ROUND(AVG(count),2)
FROM (
SELECT user_id,
    COUNT(reaction_type) AS count
FROM reactions
GROUP BY user_id
ORDER BY user_id);


 * postgresql://postgres:***@:5433/socialbuzz
1 rows affected.


round
49.05


In [22]:
%%sql 
SELECT user_id, COUNT(reaction_type) 
FROM reactions 
WHERE user_id IS NOT NULL
GROUP BY user_id
ORDER BY count DESC
LIMIT 5;

 * postgresql://postgres:***@:5433/socialbuzz
5 rows affected.


user_id,count
68724f58-bc4d-4ab0-a4e1-60cdd5e95e7d,65
c76c3393-88e2-47b0-ac37-dc4f2053f5a5,65
0871bb31-3d6e-4e4c-ab19-95a262cac0d4,63
4fe1900d-5e78-41a3-88ed-18e6889c6c77,62
d1a89d23-7d17-4949-9e1a-637317141f3d,62


#### Time-based Analysis

- Analyze the popularity trends over time.
- Break down the popularity scores and reactions by weeks or months to identify any patterns or trends.


Both the Aggregated Popularity score and the reaction count reached their peak in May and hit their lowest point in February.

In [23]:
%%sql 
SELECT EXTRACT (MONTH FROM datetime) AS month,
SUM(score) AS popularity_score,
ROUND(
((SUM(score)) - (
SELECT AVG(popularity_score)
FROM (
SELECT EXTRACT(MONTH FROM datetime) AS month,
    SUM(score) AS popularity_score
FROM reactions
JOIN reaction_types
ON reactions.reaction_type = reaction_types.reaction_type
GROUP BY month
ORDER BY month)
)) / (SELECT STDDEV(popularity_score)
FROM (
SELECT EXTRACT(MONTH FROM datetime) AS month,
    SUM(score) AS popularity_score
FROM reactions
JOIN reaction_types
ON reactions.reaction_type = reaction_types.reaction_type
GROUP BY month
ORDER BY month)),3) AS normalized_score,

COUNT(*) AS reaction_count,

ROUND(
((COUNT(*)) - (
SELECT AVG(reaction_count)
FROM (
SELECT EXTRACT(MONTH FROM datetime) AS month,
    COUNT(*) AS reaction_count
FROM reactions
JOIN reaction_types
ON reactions.reaction_type = reaction_types.reaction_type
GROUP BY month
ORDER BY month)
)) / (SELECT STDDEV(reaction_count)
FROM (
SELECT EXTRACT(MONTH FROM datetime) AS month,
    COUNT(*) AS reaction_count
FROM reactions
JOIN reaction_types
ON reactions.reaction_type = reaction_types.reaction_type
GROUP BY month
ORDER BY month)),3) AS normalized_count



FROM reactions
JOIN reaction_types
ON reactions.reaction_type = reaction_types.reaction_type
GROUP BY month
ORDER BY month;

 * postgresql://postgres:***@:5433/socialbuzz
12 rows affected.


month,popularity_score,normalized_score,reaction_count,normalized_count
1,82938,0.661,2126,1.193
2,75509,-2.066,1914,-2.039
3,78915,-0.816,2012,-0.545
4,78598,-0.932,1974,-1.124
5,86293,1.892,2138,1.376
6,80369,-0.282,2021,-0.408
7,82464,0.487,2070,0.339
8,82176,0.381,2114,1.010
9,80077,-0.389,2022,-0.392
10,82810,0.614,2056,0.126
